# Explorando datos mundiales de COVID-19

In [ ]:
import pandas as pd

### Preparación de datos

Almacenaremos los datos de casos confirmados en un dataframe. Usamos el nombre del país como índice:

In [ ]:
df_confirmados = pd.read_csv('./data/covid19_confirmados.csv',
                             index_col='Country/Region')

In [ ]:
df_confirmados.head(5)

Interpretado las columnas de fechas como Timestamp:

In [ ]:
# TO-DO

Eliminamos las columnas "Lat" y "Long":

In [ ]:
df_confirmados.drop(labels=['Lat', 'Long'], axis=1, inplace=True)

In [ ]:
df_confirmados.head(5)